In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import xlnet
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import model_utils

In [3]:
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.v9.model')

True

In [4]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='output-model/config.json')

In [5]:
xlnet_config.__dict__

{'keys': ['n_layer',
  'd_model',
  'n_head',
  'd_head',
  'd_inner',
  'ff_activation',
  'untie_r',
  'n_token'],
 'n_layer': 12,
 'd_model': 768,
 'n_head': 12,
 'd_head': 64,
 'd_inner': 3072,
 'ff_activation': 'gelu',
 'untie_r': True,
 'n_token': 32000}

In [6]:
xlnet_config.n_token

32000

In [7]:
class Model:
    def __init__(
        self,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        
        output = xlnet_model.get_sequence_output()
        lookup_table = xlnet_model.get_embedding_table()
        
        with tf.variable_scope("model", reuse=tf.AUTO_REUSE):
            with tf.variable_scope('lm_loss'):
                softmax_w = lookup_table
                softmax_b = tf.get_variable('bias', [xlnet_config.n_token], dtype=output.dtype,
                                    initializer=tf.zeros_initializer())
                logits = tf.einsum('ibd,nd->ibn', output, lookup_table) + softmax_b

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())




INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.


In [15]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [11]:
tvars = tf.trainable_variables()

assignment_map = {}
initialized_variable_names = {}

name_to_variable = collections.OrderedDict()
for var in tvars:
    name = var.name
    m = re.match('^(.*):\\d+$', name)
    if m is not None:
        name = m.group(1)
    name_to_variable[name] = var
    
name_to_variable

OrderedDict([('model/transformer/r_w_bias',
              <tf.Variable 'model/transformer/r_w_bias:0' shape=(12, 12, 64) dtype=float32_ref>),
             ('model/transformer/r_r_bias',
              <tf.Variable 'model/transformer/r_r_bias:0' shape=(12, 12, 64) dtype=float32_ref>),
             ('model/transformer/word_embedding/lookup_table',
              <tf.Variable 'model/transformer/word_embedding/lookup_table:0' shape=(32000, 768) dtype=float32_ref>),
             ('model/transformer/r_s_bias',
              <tf.Variable 'model/transformer/r_s_bias:0' shape=(12, 12, 64) dtype=float32_ref>),
             ('model/transformer/seg_embed',
              <tf.Variable 'model/transformer/seg_embed:0' shape=(12, 2, 12, 64) dtype=float32_ref>),
             ('model/transformer/layer_0/rel_attn/q/kernel',
              <tf.Variable 'model/transformer/layer_0/rel_attn/q/kernel:0' shape=(768, 12, 64) dtype=float32_ref>),
             ('model/transformer/layer_0/rel_attn/k/kernel',
         

In [13]:
checkpoint = 'output-model/model.ckpt-300000'
tf.train.list_variables(checkpoint)

[('beta1_power', []),
 ('beta2_power', []),
 ('global_step', []),
 ('model/lm_loss/bias', [32000]),
 ('model/lm_loss/bias/Adam', [32000]),
 ('model/lm_loss/bias/Adam_1', [32000]),
 ('model/transformer/layer_0/ff/LayerNorm/beta', [768]),
 ('model/transformer/layer_0/ff/LayerNorm/beta/Adam', [768]),
 ('model/transformer/layer_0/ff/LayerNorm/beta/Adam_1', [768]),
 ('model/transformer/layer_0/ff/LayerNorm/gamma', [768]),
 ('model/transformer/layer_0/ff/LayerNorm/gamma/Adam', [768]),
 ('model/transformer/layer_0/ff/LayerNorm/gamma/Adam_1', [768]),
 ('model/transformer/layer_0/ff/layer_1/bias', [3072]),
 ('model/transformer/layer_0/ff/layer_1/bias/Adam', [3072]),
 ('model/transformer/layer_0/ff/layer_1/bias/Adam_1', [3072]),
 ('model/transformer/layer_0/ff/layer_1/kernel', [768, 3072]),
 ('model/transformer/layer_0/ff/layer_1/kernel/Adam', [768, 3072]),
 ('model/transformer/layer_0/ff/layer_1/kernel/Adam_1', [768, 3072]),
 ('model/transformer/layer_0/ff/layer_2/bias', [768]),
 ('model/transf

In [16]:
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [18]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from output-model/model.ckpt-300000


In [19]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'xlnet-base/model.ckpt')

'xlnet-base/model.ckpt'